In [6]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain_openai import OpenAI, ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from pprint import pprint
from dotenv import load_dotenv
load_dotenv() 

from src.utils import get_vector_store

MODEL = "gpt-4.1-nano" # "gpt-4.1-mini", "gpt-3.5-turbo"
TEMPERATURE = 0.5
SYSTEM_PROMPT = """
You are a helpful AI assistant designed to accurately answer questions related to Magic The Gathering (MTG) cards. You will receive an input question and potential cards that are relevant to the question. Your task is to provide a concise, accurate, and wholistic answer to the question. You may not need to return all cards or all the related metadata for each card in the context, but you should use the provided context to inform your answer. If the question is not related to MTG cards, you should respond with "I don't know" or "I cannot answer that question."

## Question
{user_input}

## Context
{context}
"""

def get_agent(user_input: str, context: str = "") -> str:
    """Create and return an agent executor for multi-turn LLM agent."""
    llm = ChatOpenAI(
        model=MODEL, 
        temperature=TEMPERATURE, 
    )
    agent_prompt = PromptTemplate(
        input_variables=['user_input', 'context'], 
        template=SYSTEM_PROMPT 
    )
    chain = LLMChain(llm=llm, prompt=agent_prompt)
    response = chain.run(user_input=user_input, context=context)
    return response

def get_context(user_input: str, K: int = 100, feats: list = ['name', 'text', 'type', 'power', 'toughness', 'manaCost', 'colorIdentity']) -> str:
    vectorstore = get_vector_store()
    results = vectorstore.similarity_search_with_score(
        user_input, 
        k=K, 
    )
    
    # filter for results & merge metadata
    context = '|'.join(feats) + '\n' + '\n'.join(
        ['|'.join(
                [result[0].metadata[k] if k in result[0].metadata else '' for k in feats]
            ) for result in results]
        )
    return context



In [7]:
user_input = "Find me creatures that make squirrels."

In [8]:
context = get_context(user_input)
response = get_agent(user_input, context)
print(response)

/var/folders/t3/_06vpls11x13y6v4y5325lhr0000gn/T/ipykernel_6313/2792319249.py:34: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=agent_prompt)
/var/folders/t3/_06vpls11x13y6v4y5325lhr0000gn/T/ipykernel_6313/2792319249.py:35: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(user_input=user_input, context=context)


The following MTG creatures from your list make Squirrels:

1. **Drey Keeper** — When it enters, create two 1/1 green Squirrel creature tokens.
2. **Squirrel Mob** — Gets +1/+1 for each other Squirrel on the battlefield.
3. **Squirrel Squatters** — When it attacks, create a tapped 1/1 green Squirrel token for each Attraction visited this turn.
4. **Nut Collector** — At the beginning of your upkeep, creates a 1/1 green Squirrel token.
5. **Earl of Squirrel** — Squirrellink; other Squirrels you control get +1/+1 and damage dealt by Squirrels creates Squirrel tokens.
6. **Chittering Doom** — Creates a 1/1 green Squirrel token whenever you roll a 4 or higher on a die.
7. **Squirrel Wrangler** — Sacrifice a land to create two Squirrel tokens or give Squirrels +1/+1 until end of turn.
8. **Chitterspitter** — Taps to create a 1/1 green Squirrel token.
9. **Acornelia, Fashionable Filcher** — Creates a 1/1 green Squirrel token whenever a Squirrel enters or dies.
10. **Deep Forest Hermit** — Whe

In [11]:
vectorstore = get_vector_store()
results = vectorstore.similarity_search_with_score(
    user_input, 
    k=3, 
)

In [16]:
sorted(list(results[0][0].metadata.keys()))

['cardName',
 'colorIdentity',
 'colors',
 'convertedManaCost',
 'firstPrinting',
 'foreignData',
 'hasAlternativeDeckLimit',
 'identifiers.scryfallOracleId',
 'isFunny',
 'isReserved',
 'layout',
 'leadershipSkills.brawl',
 'leadershipSkills.commander',
 'leadershipSkills.oathbreaker',
 'manaCost',
 'manaValue',
 'name',
 'power',
 'printings',
 'purchaseUrls.cardKingdom',
 'purchaseUrls.cardKingdomFoil',
 'purchaseUrls.cardmarket',
 'purchaseUrls.tcgplayer',
 'rulings',
 'subtypes',
 'text',
 'toughness',
 'type',
 'types']

In [17]:
from src.utils import load_json_file

data = load_json_file()
data

/Users/ccoggins/Documents/Code/GitHub/ai-cookbook/apps/mtg_recommender/src/utils.py:81: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False)
/Users/ccoggins/Documents/Code/GitHub/ai-cookbook/apps/mtg_recommender/src/utils.py:81: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False)


,cardName,identifiers.scryfallOracleId,colorIdentity,colors,manaCost,convertedManaCost,manaValue,layout,side,faceName,...,edhrecSaltiness,firstPrinting,printings,foreignData,hand,isFunny,isReserved,life,asciiName,name
0,"""Ach! Hans, Run!""",a2c5ee76-6084-413c-bb70-45490d818374,"G, R","G, R",{2}{R}{R}{G}{G},6.0,6.0,normal,None,None,...,NaN,UNH,UNH,[],None,True,False,None,None,"""Ach! Hans, Run!"""
1,"""Brims"" Barone, Midway Mobster",c64c31f2-c1be-414e-9dff-c3b77ba97545,"B, W","B, W",{3}{W}{B},5.0,5.0,normal,None,None,...,NaN,UNF,UNF,[],None,True,False,None,None,"""Brims"" Barone, Midway Mobster"
2,"""Lifetime"" Pass Holder",7bf6f13a-3c90-4bda-bc84-e026828bf4d1,B,B,{B},1.0,1.0,normal,None,None,...,0.50,UNF,UNF,[],None,False,False,None,None,"""Lifetime"" Pass Holder"
3,"""Name Sticker"" Goblin",acee1d16-1651-4e2c-8138-cc6456c4ee71,R,R,{2}{R},3.0,3.0,normal,None,None,...,NaN,UNF,UNF,[],None,False,False,None,None,"""Name Sticker"" Goblin"
4,"""Rumors of My Death . . .""",38bcba8b-2838-4ac8-9976-f9ccaa94fdba,B,B,{2}{B},3.0,3.0,normal,None,None,...,NaN,UST,UST,[],None,True,False,None,None,"""Rumors of My Death . . ."""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32793,"Éomer, King of Rohan",4f8191e6-a6aa-4426-9014-eff4e0efee9e,"R, W","R, W",{3}{R}{W},5.0,5.0,normal,None,None,...,0.32,LTC,LTC,"[{'language': 'German', 'name': 'Éomer, König ...",None,False,False,None,"Eomer, King of Rohan","Éomer, King of Rohan"
32794,"Éomer, Marshal of Rohan",b2d95950-18b3-463f-94f4-299e420751dc,R,R,{2}{R}{R},4.0,4.0,normal,None,None,...,0.22,LTR,"LTR, PLTR","[{'language': 'German', 'name': 'Éomer, Marsch...",None,False,False,None,"Eomer, Marshal of Rohan","Éomer, Marshal of Rohan"
32795,"Éowyn, Fearless Knight",dcc349e9-399d-4f33-8131-d1801272466a,"R, W","R, W",{2}{R}{W},4.0,4.0,normal,None,None,...,0.33,LTR,"LTR, PLTR","[{'language': 'German', 'name': 'Éowyn, furcht...",None,False,False,None,"Eowyn, Fearless Knight","Éowyn, Fearless Knight"
32796,"Éowyn, Lady of Rohan",1ee3753c-3b6e-4182-9305-2ab757f485f0,W,W,{2}{W},3.0,3.0,normal,None,None,...,0.11,LTR,LTR,"[{'language': 'German', 'name': 'Éowyn, Herrin...",None,False,False,None,"Eowyn, Lady of Rohan","Éowyn, Lady of Rohan"


In [18]:
sorted(data.columns.tolist())

['asciiName',
 'cardName',
 'colorIdentity',
 'colorIndicator',
 'colors',
 'convertedManaCost',
 'defense',
 'edhrecRank',
 'edhrecSaltiness',
 'faceConvertedManaCost',
 'faceManaValue',
 'faceName',
 'firstPrinting',
 'foreignData',
 'hand',
 'hasAlternativeDeckLimit',
 'identifiers.scryfallOracleId',
 'isFunny',
 'isReserved',
 'keywords',
 'layout',
 'leadershipSkills.brawl',
 'leadershipSkills.commander',
 'leadershipSkills.oathbreaker',
 'legalities.alchemy',
 'legalities.brawl',
 'legalities.commander',
 'legalities.duel',
 'legalities.explorer',
 'legalities.future',
 'legalities.gladiator',
 'legalities.historic',
 'legalities.legacy',
 'legalities.modern',
 'legalities.oathbreaker',
 'legalities.oldschool',
 'legalities.pauper',
 'legalities.paupercommander',
 'legalities.penny',
 'legalities.pioneer',
 'legalities.predh',
 'legalities.premodern',
 'legalities.standard',
 'legalities.standardbrawl',
 'legalities.timeless',
 'legalities.vintage',
 'life',
 'loyalty',
 'manaCos

In [26]:
data.loc[data['cardName'].str.contains('Emrakul and')]

,cardName,identifiers.scryfallOracleId,colorIdentity,colors,manaCost,convertedManaCost,manaValue,layout,side,faceName,...,edhrecSaltiness,firstPrinting,printings,foreignData,hand,isFunny,isReserved,life,asciiName,name
8536,Emrakul and Chatterfang,3c51bc3d-9f98-4d91-bb7b-a44d48ea06f9,"B, G",None,{8}{G}{G}{G},11.0,11.0,normal,None,None,...,NaN,UNK,UNK,[],None,True,False,None,None,Emrakul and Chatterfang


In [29]:
data['legalities.commander'].value_counts()

legalities.commander
Legal     29533
Banned       86
Name: count, dtype: int64